# Automatic tuning of structured controllers in JuliaSim

## JuliaSim
A platform for modeling, simulation and *control*

## Problem setting
- We want to control something 😄
- The controller is *structured*
- Design criteria may be provided


## How do we solve it?
- Build a model using ModelingToolkit
- Add *analysis points* to the model (signal names)
- Specify *tuning objectives*
- Solve an optimization problem

## Today's example
System model is an electrical motor with a cascaded PID controller for position and velocity control.
![Block diagram](https://help.juliahub.com/DyadControlSystems/dev/figs/cascade_pid.png)

In [ ]:
using Pkg, Revise
Pkg.precompile()
using DyadControlSystems.MPC

## What is an AnalysisPoint?
Think of it as a *named signal*

In [ ]:
using ModelingToolkit
using ModelingToolkitStandardLibrary.Blocks

@named input = RealInput()
@named output = RealOutput()

connect(output, input)

In [ ]:
connect(output, :name, input)

In [ ]:
using DyadControlSystems

sys = DyadControlSystems.ControlDemoSystems.dcmotor(k=2)

In [ ]:
```              d
     ┌─────┐  │  ┌─────┐
r  e │     │u ▼  │     │ y
──+─►│  C  ├──+─►│  P  ├─┬─►
  ▲  │     │     │     │ │
 -│  └─────┘     └─────┘ │
  │                      │
  └──────────────────────┘
```
equations(sys)

## Specify tunable parameters and operating points
- Tunable parameters will be optimized
    - Any parameter can be tunable (not only controller parameters)
- Operating points determine where *linearization* is performed
- An operating point include states, inputs *and parameters*

In [ ]:
sysc = complete(sys)
tunable_parameters = [
    sysc.pi_controller.gainPI.k => (1e-9, 100.0) # parameter => (lower_bound, upper_bound)
    sysc.pi_controller.int.k    => (2.0, 1e2)
]

operating_points = [ # Can be one or several operating points
    ModelingToolkit.defaults(sys)
]

This system is linear, one operating point is sufficient for now

## Specify tuning objectives
What do we want to achieve?
- Accurate **reference tracking**?
- Short **settling time**?
- Limit **overshoot**?
- Fast **disturbance rejection**?
- Large **gain and phase margins**?
- Limited **noise amplification**?
- **Robustness** w.r.t. model uncertainty?
- All of the above?


#### Reference tracking
Specify a reference model

In [ ]:
ω     = 2pi*20.0
ζ     = 0.8
Gref  = tf(ω^2, [1, 2ζ*ω, ω^2])
sto   = StepTrackingObjective(reference_model=Gref, tolerance=0.2, input=:r, output=:y)
plot(sto, 0.4, size=(600, 400))

#### Sensitivity objectives
Shape the *sensitivity function*---promote both **performance** and **robustness** 
```              d
     ┌─────┐  │  ┌─────┐
r  e │     │u ▼  │     │ y
──+─►│  C  ├──+─►│  P  ├─┬─►
  ▲  │     │     │     │ │
 -│  └─────┘     └─────┘ │
  │                      │
  └──────────────────────┘
```

$$
\begin{aligned}
S(s) &= \dfrac{1}{I + P(s)C(s)} \\
e &= Sr \\
M_S &= ||S(s)||_\infty \\
\phi_m &≥ 2 \sin^{-1}\left(\dfrac{1}{2M_S}\right) \text{rad}\\
g_m &≥ \dfrac{M_S}{M_S-1}
\end{aligned}
$$

The *peak of the sensitivity function*, $M_S$, must not be too high!

In [ ]:
WS    = tf([1.5, 0], [1, 50])
mso   = MaximumSensitivityObjective(WS, :y)
plot(mso, size=(600, 400))

## Other classical design criteria
- Limit overshoot
- Desired rise time above some % of final value
- Specified settling time to within some % of final value

In [ ]:
oo = OvershootObjective(max_value = 1.1, input=:r, output=:y)

In [ ]:
rto = RiseTimeObjective(min_value = 0.9, time = 0.025, input=:r, output=:y)
plot(rto, xlims=(0, 0.1), ylims=(0, 2), size=(600, 400))

In [ ]:
seto = SettlingTimeObjective(; final_value = 1.0, time = 0.025, tolerance = 0.10, input=:r, output=:y) 
plot(seto, xlims=(0, 0.1), ylims=(0, 2), size=(600, 400))

#### Package objectives together

In [ ]:
objectives = [
    sto,    # Step tracking 
    mso,    # Maximum sensitivity
    seto,   # Settling time
]

## Create a `StructuredAutoTuningProblem`

In [ ]:
w = exp10.(LinRange(0, 3, 200)) # Frequency vector
t = 0:0.001:0.21;               # Time vector

In [ ]:
prob1 = StructuredAutoTuningProblem(sys, w, t, objectives, operating_points, tunable_parameters)

In [ ]:
plot(prob1)

#### Solve

In [ ]:
p0 = [2.0, 20] # Initial guess
res1 = solve(prob1, p0,
    MPC.IpoptSolver(verbose=true, exact_hessian=false, acceptable_iter=4, tol=1e-4, acceptable_tol=1e-3, max_iter=100);
)

In [ ]:
plot(res1)

In [ ]:
res1.objective_status[1] # Inspect the results in the first (and in this case only) operating point

### Add the outer position loop
![Block diagram](https://help.juliahub.com/DyadControlSystems/dev/figs/cascade_pid.png)

In [ ]:
sys_inner             = DyadControlSystems.ControlDemoSystems.dcmotor(ref=nothing)
@named ref            = Blocks.Step(height = 1, start_time = 0)
@named ref_diff       = Blocks.Derivative(T=0.1) # This will differentiate q_ref to q̇_ref
@named add            = Blocks.Add()      # The middle ∑ block in the diagram
@named p_controller   = Blocks.Gain(10.0) # Kₚ
@named outer_feedback = Blocks.Feedback() # The leftmost ∑ block in the diagram
@named id             = Blocks.Gain(1.0)  # a trivial identity element to allow us to place the analysis point :r in the right spot

#connect = ModelingToolkit.connect
connections = [
    connect(ref.output, :r, id.input)                               # We now place analysis point :r here
    connect(id.output, outer_feedback.input1, ref_diff.input)
    connect(ref_diff.output, add.input1)
    connect(add.output, sys_inner.feedback.input1)
    connect(p_controller.output, :up, add.input2)                   # Analysis point :up
    connect(sys_inner.angle_sensor.phi, :yp, outer_feedback.input2) # Analysis point :yp
    connect(outer_feedback.output, :ep, p_controller.input)         # Analysis point :ep
]

@named closed_loop = ODESystem(connections, ModelingToolkit.get_iv(sys_inner); systems = [sys_inner, ref, id, ref_diff, add, p_controller, outer_feedback])

In [ ]:
cl = complete(closed_loop)

tunable_parameters = [
    cl.dcmotor.pi_controller.gainPI.k => (1e-1, 10.0)
    cl.dcmotor.pi_controller.int.k    => (2.0, 1e2)
    cl.p_controller.k                 => (1e-2, 1e2)
]

operating_points = [ # Can be one or several operating points
    ModelingToolkit.defaults(closed_loop)
]

ωp    = 2pi*2.0                        # Desired position-loop bandwidth
ζp    = 0.8
Pref  = tf(ωp^2, [1, 2ζp*ωp, ωp^2])    # Desired position step response
stp   = StepTrackingObjective(reference_model = Pref, tolerance = 0.05, input=:r, output=:yp)
mso2  = MaximumSensitivityObjective(weight=WS, output=:dcmotor_y, loop_openings=[:yp])
objectives = [
    stp,
    mso2,
]

t = 0:0.001:1
prob2 = DyadControlSystems.StructuredAutoTuningProblem(cl, w, t, objectives, operating_points, tunable_parameters)

p0 = [1, 20, 0.1]
res2 = solve(prob2, p0,
    MPC.IpoptSolver(verbose=false, exact_hessian=false, acceptable_iter=4, tol=1e-3, acceptable_tol=1e-2, max_iter=100);
)

In [ ]:
plot(res2)

## Tuning under parametric uncertainty

In [ ]:
using MonteCarloMeasurements
N = 5 # Number of samples for the uncertain parameters
J = Particles(N, Uniform(0.015, 0.025))

In [ ]:
(J+2)^2 # J behaves like a regular floating-point number, but represents an uncertain number

In [ ]:
plot(J, bins=N, size=(400, 300))

In [ ]:
sys = DyadControlSystems.ControlDemoSystems.dcmotor()
sysc = complete(sys)

opu = ModelingToolkit.defaults(sys)
opu[sysc.inertia.J] = J
operating_pointsu = [opu];

tunable_parameters = [
    sysc.pi_controller.gainPI.k => (1e-9, 100.0)
    sysc.pi_controller.int.k    => (2.0, 1e2)
]

objectives = [
    sto,
    seto,
    mso,
]

t = 0:0.001:0.21
prob3 = StructuredAutoTuningProblem(sysc, w, t, objectives, operating_pointsu, tunable_parameters)

p0 = [1, 20]
res3 = solve(prob3, p0,
    MPC.IpoptSolver(verbose=false, exact_hessian=false, acceptable_iter=4, tol=1e-3, acceptable_tol=1e-2, max_iter=100);
)

In [ ]:
plot(res3)

## Summary
- Automatic controller tuning in ModelingToolkit
- Meet objectives and specifications
- Model uncertain parameters

## See also
- Youtube video series on control in Julia


## Additional material
### Limiting noise amplification
Include objective limiting the transfer function $CS$
$$u = \dfrac{C}{I + PC}n$$

In [ ]:
mto = MaximumTransferObjective(tf(1), :y, :u)

In [ ]:
objectives = [mso, mto]
tunable_parameters = [
    sysc.pi_controller.gainPI.k => (1e-9, 100.0) # parameter => (lower_bound, upper_bound)
    sysc.pi_controller.int.k    => (2.0, 1e2)
]

operating_points = [ModelingToolkit.defaults(sys)]

prob4 = StructuredAutoTuningProblem(sys, w, t, objectives, operating_points, tunable_parameters)
plot(prob4)
res4 = solve(prob4, p0,
    MPC.IpoptSolver(verbose=false, exact_hessian=false, acceptable_iter=4, tol=1e-3, acceptable_tol=1e-2, max_iter=100);
)
plot(res4)

In [ ]:
res4